In [1]:
import numpy as np
from utils_clean import generate_sample, DEOptim, RandomOptim, PSOOptim, visualize_meshgrid, get_likelihood_grids, visualize_pred_meshgrid
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import plotly.express as px
import pandas as pd

In [2]:
target_func = lambda X: (20 + X**2 - 10 * np.cos(2*np.pi*X)).sum(axis=0)

n_dims = 2
lower_bound = -2.5, -1.5
upper_bound = +1.5, +2.5

In [3]:
de = DEOptim(visualize=False, maxiter=100, popsize=10)

In [4]:
rs = RandomOptim(maxiter=1000, visualize=False)

In [5]:
seed = np.random.RandomState(42)
part = 10
dim = 2
pso = PSOOptim(n_iters=100, n_particles=part, c1=0.5, c2=0.3, w=0.9, init_pos=(seed.rand(part, dim)*22)-11, visualize=False)

In [6]:
results_dict = {
    "optim": [],
    "n": [],
    "log_likelihood": [],
    "MSE": []
}

In [7]:

for n in [10, 50, 200, 500, 1000, 5000, 10000]:
    for optim in [de, rs, pso]:
        gpr = GaussianProcessRegressor(optimizer=optim.optimize, kernel=RBF(length_scale=(1,1)))
        X, y = generate_sample(70, n_dims, lower_bound, upper_bound, target_func)
        gpr.fit(X.T, y[...,None])
        results_dict["optim"].append(optim.__class__.__name__)
        results_dict["n"].append(n)
        results_dict["log_likelihood"].append(gpr.log_marginal_likelihood_value_)
        results_dict["MSE"].append(mean_squared_error(y, gpr.predict(X.T)))

2022-08-07 16:23:46,911 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=3.41e+4
2022-08-07 16:23:48,834 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 34147.067725817666, best pos: [-1.03823026 -1.09540646]
2022-08-07 16:23:52,488 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=3.41e+4
2022-08-07 16:23:54,243 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 34147.26360634334, best pos: [-1.04240325 -1.0953739 ]
2022-08-07 16:23:56,978 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=3.41e+4
2022-08-07 16:23:58,314 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 34147.0

In [8]:
df = pd.DataFrame(results_dict)

In [9]:
df

,optim,n,log_likelihood,MSE
0,DEOptim,10,-34146.927641,9.980445e-17
1,RandomOptim,10,-40263.110418,1.983151e-17
2,PSOOptim,10,-34147.067726,1.007532e-16
3,DEOptim,50,-34146.927641,9.980440e-17
4,RandomOptim,50,-40263.110418,1.983151e-17
5,PSOOptim,50,-34147.263606,9.928733e-17
6,DEOptim,200,-34146.927641,9.980446e-17
7,RandomOptim,200,-40263.110418,1.983151e-17
8,PSOOptim,200,-34147.023497,9.930356e-17
9,DEOptim,500,-34146.927641,9.980441e-17


In [10]:
px.line(df, x="n", y="log_likelihood", color="optim")

In [11]:
px.line(df, x="n", y="MSE", color="optim")